# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     Miguel Aaron Castillon Ochoa|   |
| **Fecha**     09/11/2025 |   |
| **Expediente*751858* |   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

## Librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

## Cargar dataset

In [2]:
df = pd.read_csv("Advertising.csv")
print(df.head())

   Unnamed: 0     TV  radio  newspaper  sales
0           1  230.1   37.8       69.2   22.1
1           2   44.5   39.3       45.1   10.4
2           3   17.2   45.9       69.3    9.3
3           4  151.5   41.3       58.5   18.5
4           5  180.8   10.8       58.4   12.9


## Definir variables

In [3]:
X = df[["TV", "radio", "newspaper"]]
y = df["sales"]

## Train y test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
resultados = {}

## Modelo OLS

In [6]:
ols = LinearRegression()
ols.fit(X_train, y_train)
y_pred_ols = ols.predict(X_test)

resultados["OLS"] = {
    "Coeficientes": ols.coef_,
    "Intercepto": ols.intercept_,
    "R2": r2_score(y_test, y_pred_ols),
    "MSE": mean_squared_error(y_test, y_pred_ols)
}

## Modelo Ridge

In [7]:
ridge = Ridge(alpha=1.0)  # puedes ajustar alpha
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

resultados["Ridge"] = {
    "Coeficientes": ridge.coef_,
    "Intercepto": ridge.intercept_,
    "R2": r2_score(y_test, y_pred_ridge),
    "MSE": mean_squared_error(y_test, y_pred_ridge)
}

## Modelo Lasso

In [9]:
lasso = Lasso(alpha=0.1)  # puedes ajustar alpha
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

resultados["Lasso"] = {
    "Coeficientes": lasso.coef_,
    "Intercepto": lasso.intercept_,
    "R2": r2_score(y_test, y_pred_lasso),
    "MSE": mean_squared_error(y_test, y_pred_lasso)
}

## Resultados

In [10]:
for modelo, valores in resultados.items():
    print("\nModelo:", modelo)
    print("Coeficientes:", valores["Coeficientes"])
    print("Intercepto:", valores["Intercepto"])
    print("R2:", round(valores["R2"], 4))
    print("MSE:", round(valores["MSE"], 4))


Modelo: OLS
Coeficientes: [0.04472952 0.18919505 0.00276111]
Intercepto: 2.979067338122629
R2: 0.8994
MSE: 3.1741

Modelo: Ridge
Coeficientes: [0.04472953 0.18918869 0.00276287]
Intercepto: 2.97915853521409
R2: 0.8994
MSE: 3.1741

Modelo: Lasso
Coeficientes: [0.04471961 0.18881106 0.00262707]
Intercepto: 2.993352005331829
R2: 0.8996
MSE: 3.1705


## Comparación

#### Precisión (R² y MSE):

Los tres modelos explican ~90% de la variabilidad de las ventas, con errores casi iguales.

Lasso tiene un rendimiento apenas superior (menor MSE y mayor R²).

#### Efecto en los coeficientes:

OLS y Ridge producen coeficientes muy similares, ya que Ridge solo penaliza si los valores de los parámetros son muy grandes (aquí no es el caso).

Lasso empuja más hacia valores pequeños, especialmente en la variable Newspaper, que ya de por sí aporta muy poco al modelo (coeficiente cercano a 0).

#### Interpretación práctica:

En este dataset, TV y Radio son los predictores más importantes.

Newspaper tiene una influencia casi nula en las ventas.

Usar Lasso puede ser más útil cuando queremos simplificar el modelo y reducir variables irrelevantes.

## Conclusion

Los tres modelos son buenos, pero Lasso ofrece la mejor combinación de simplicidad y precisión en este caso. Ridge no aporta mejoras porque los coeficientes no son extremos. OLS sigue siendo perfectamente válido como baseline.